In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
from pprint import pprint

from src.supabase_client import get_user, list_users
from tests.common import run_new_training_week_process_wrapped, run_mid_week_update_process_wrapped


# Test: One Offs

In [ ]:
mock_user = get_user(105359460)
response = run_new_training_week_process_wrapped(mock_user)
print(response)

In [ ]:
mock_user = get_user(os.environ["JAMIES_ATHLETE_ID"])
response = run_mid_week_update_process_wrapped(mock_user)
print(response)

# Evals: New Training Week

In [ ]:
path = 'tests/artifacts/new_training_week_errors'
files = os.listdir(path)

for file in sorted(files):
    print(f"Loading {file}")
    with open(os.path.join(path, file), "r") as f:
        results = json.load(f)
    print(sum(result['error'] for result in results) / len(results))

In [ ]:
for result in results:
    print(result['user']['email'])
    print("target:", result['recommended_weekly_mileage'])
    print("actual:", result['generated_weekly_mileage'])
    print("error:", result['error'])
    print(result['planning'])
    print("." * 100)